<a href="https://colab.research.google.com/github/omarahmed999/Rnnsentimentassignment/blob/main/using_sentiment140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [5]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00


In [6]:
from google.colab import files
files.upload()  # Select the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"omarahmed007","key":"ccfb07265aae624d4d54c4bde60e25ff"}'}

In [7]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Secure the file

In [8]:
!kaggle datasets download -d kazanova/sentiment140
!unzip sentiment140.zip

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout

# Load Sentiment140 dataset (adjust path if needed)
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin-1", header=None)
df = df[[0, 5]]  # Selecting sentiment and text columns
df.columns = ["sentiment", "text"]

df["sentiment"] = df["sentiment"].map({0: 0, 4: 1})  # Convert sentiment labels to binary

# Splitting data
train_size = int(0.8 * len(df))
train_texts, train_labels = df["text"][:train_size], df["sentiment"][:train_size]
test_texts, test_labels = df["text"][train_size:], df["sentiment"][train_size:]

# Tokenization
vocab_size = 20000  # Vocabulary size
max_length = 100  # Max sequence length
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Building the RNN Model
model = Sequential([
    Embedding(vocab_size, 128, input_length=max_length),
    SimpleRNN(64, return_sequences=True),
    SimpleRNN(32),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(train_padded, train_labels, epochs=5, batch_size=128, validation_data=(test_padded, test_labels))

# Evaluate on test set
loss, acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {acc * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 1063s 106ms/step - accuracy: 0.6241 - loss: 0.6538 - val_accuracy: 0.0000e+00 - val_loss: 0.9654
Epoch 2/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 1057s 106ms/step - accuracy: 0.6248 - loss: 0.6461 - val_accuracy: 0.0000e+00 - val_loss: 0.9674
Epoch 3/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 1059s 106ms/step - accuracy: 0.6253 - loss: 0.6482 - val_accuracy: 0.0000e+00 - val_loss: 0.9519
Epoch 4/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 1057s 106ms/step - accuracy: 0.6252 - loss: 0.6541 - val_accuracy: 0.0000e+00 - val_loss: 0.9737
Epoch 5/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 1058s 106ms/step - accuracy: 0.6252 - loss: 0.6544 - val_accuracy: 0.0000e+00 - val_loss: 1.0091
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 105s 10ms/step - accuracy: 0.0000e+00 - loss: 1.0089
Test Accuracy: 0.00%


In [11]:
model.save("sentiment_rnn_model.keras")  # Saves the model in HDF5 format


In [12]:
new_text = ["I love this movie! It was amazing."]

# Tokenize and pad the input sentence
new_sequence = tokenizer.texts_to_sequences(new_text)
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')



In [15]:
# Predict sentiment (output will be a probability)
prediction = model.predict(new_padded)[0][0]

# Interpret the prediction
if prediction < 0.5:
    print("Sentiment: Positive 😀", f"({prediction:.2f})")
else:
    print("Sentiment: Negative 😞", f"({prediction:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Sentiment: Positive 😀 (0.36)
